In [1]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [2]:
MAX_COST = 8.9
MAX_TIME = 0.89
DIR = DATABASE_DIR
# MAX_COST = 100.0
# MAX_TIME = 10.0
# DIR = (MAIN_DIR / "_archive" / "experiments" / "2025-01-27" / "600" / "database")

generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST_600" / "index.json")

id_to_generator_name = {
    instance.id(): instance.filepath.parts[-2] for instance in test_instances
}

id_to_name = {
    instance.id(): instance.filepath.parts[-1] for instance in test_instances
}

In [3]:
def agg_prefix(prefix):

    frames = []

    def agg_cost(x):
        count = x.shape[0]
        x = x[x < MAX_COST]
        x = x.sort_values()
        idx = count // 2
        # if x.shape[0] == 0:
        #     return np.nan
        if x.shape[0] <= idx:
            return x.iloc[-1]
        return x.iloc[idx]
        

    # for idx, db_path in enumerate():
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        print(db_path)
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        series = (
            results.loc[results["prefix"].str.startswith("test")]
            .groupby(["instance_id", "prefix"])["cost"]
            .min()
            .reset_index()
            .groupby("instance_id")["cost"]
            .agg(agg_cost)
        )
        frames.append(series)
        

    df = (
        pd.concat(frames, axis=1)
        .mean(axis=1)
        .round(3)
        .rename(prefix)
        .to_frame()
    )

    df["generator"] = df.index.map(id_to_generator_name)
    df["name"] = df.index.map(id_to_name)
    df = df.pivot_table(index="generator", columns="name", values=prefix).loc[generators, :]
    return df

n = 30
plain = agg_prefix(f"run-plain-{n}")
plain = plain.assign(generator=f"plain-{n}").set_index("generator", append=True)

C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980890.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980891.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980892.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980893.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980894.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980895.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980896.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algorithm-portfolios-main\database\run-plain-30-980897.db
C:\Users\zakrz\Documents\DataScience\praca magisterska\raw-algor

In [4]:
plain_total_mean = plain.mean(axis=1).mean(axis=0)
print(f"{plain_total_mean=:.3f}")

plain_total_mean=1.173


In [5]:
df = plain.copy().sort_index(level=0).loc[generators]   
df["mean"] = df.mean(axis=1)

In [6]:
df.style.background_gradient(cmap="Reds", vmin=0, vmax=5).format("{:.3f}")

,name,000.tsp,001.tsp,002.tsp,003.tsp,004.tsp,mean
generator,generator,,,,,,
cluster_netgen,plain-30,2.288,2.519,3.372,1.584,0.819,2.116
compression,plain-30,0.556,1.671,0.904,1.119,0.845,1.019
expansion,plain-30,1.098,1.433,0.880,0.554,1.347,1.062
explosion,plain-30,0.321,0.744,1.389,0.451,0.989,0.779
grid,plain-30,0.216,0.742,3.076,0.304,1.001,1.068
cluster,plain-30,0.682,0.826,0.648,0.879,2.974,1.202
implosion,plain-30,2.126,0.690,0.478,1.534,4.770,1.920
linearprojection,plain-30,0.495,0.536,0.913,0.384,1.350,0.736
rotation,plain-30,1.736,1.076,0.765,1.699,0.514,1.158


In [7]:
df.to_excel("tmp.xlsx")

In [8]:
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0),
            "time",
        ].sum() / 3600
    return np.round(pd.Series(times).mean(), 2)

n = 30
plain = agg_prefix(f"run-plain-{n}")

print(f"plain: {plain:.2f} h")

plain: 0.75 h


In [9]:
# skip cutt-offs
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0)
            & results["cost"].lt(MAX_COST),
            "time",
        ].mean()
    return np.round(pd.Series(times).mean(), 2)

n = 30
plain = agg_prefix(f"run-plain-{n}")
print(f"plain: {plain:.2f}")

plain: 0.24


In [10]:
# % cutt-offs
def agg_prefix(prefix):
    times = {}
    for idx, db_path in enumerate(DIR.glob(f"{prefix}-*.db")):
        db = DB(db_path)
        results = pd.read_sql_query("SELECT * FROM results", db._conn)
        times[idx] = results.loc[
            results["prefix"].str.startswith("config")
            & results["cached"].eq(0)
            & results["surrogate"].eq(0),
            "cost"
        ].eq(MAX_COST).mean()
    return np.round(pd.Series(times).mean(), 2)

n = 30
plain = agg_prefix(f"run-plain-{n}")

print(f"plain: {plain:.2f}")

plain: 0.55
